<a href="https://colab.research.google.com/github/ElenaShargina/patterns/blob/master/%D0%9F%D0%BE%D0%B2%D0%B5%D0%B4%D0%B5%D0%BD%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B5%20%D0%BF%D0%B0%D1%82%D1%82%D0%B5%D1%80%D0%BD%D1%8B/Command.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Command / Команда
Инкапсулирует запрос как объект, позволяя тем самым задавать параметры клиентов для обработки соответствующих запросов, ставить запросы в очередь или протоколировать их, а также поддерживать отмену операций.

Например - кнопки и меню не знают, кто будет обрабатывать их запросы. В реализации кнопок и меню
нет действий, которые они должны инициировать.

## Пример реализации

<img src='http://feana.ru/wp-content/uploads/2023/05/command.png'>

In [10]:
# абстрактный класс комманды
class Command():
    history = []
    def __init__(self,application):
        self._app = application
    def execute(self):
        pass

# класс конкретной команды
# определяет связь между объектом-получателем команды (Document) и действием
class OpenCommand(Command):
    def execute(self):
        name = input('Enter the name of document...\n')
        # если документ не существует, создаем новый
        if hash(name) not in self._app.docs:
            self._app.add_doc(Document(name))
            print(f'New document {name} is created!')
            self._app.docs[hash(name)].rewrite()
            self._app.docs[hash(name)].show()
        # если документ существует, открываем его
        else:
            self._app.docs[hash(name)].open()
            self._app.docs[hash(name)].show()
        self._app.current_doc = self._app.docs[hash(name)]

# класс конкретной команды
class RewriteCommand(Command):
    def execute(self):
        if self._app.current_doc == None:
            print('Error: open some document!')
        else:
            self._app.current_doc.rewrite()

# класс конкретной команды
class DeleteCommand(Command):
    def execute(self):
        if self._app.current_doc == None:
            print('Error: open some document!')
        else:
            # удаляем из списка документов приложения
            self._app.delete_doc(self._app.current_doc.name)
            # удаляем сам объект документа
            self._app.current_doc.delete()
            self._app.current_doc = None

# получатель команды, располагает информацией о способах выполнения операций,
# необходимых для удовлетворения запроса
class Document:
    def __init__(self,name):
        self.name = name
        self.text = ''

    def open(self):
        print(f'Document {self.name} is opened!')

    def rewrite(self):
        text = input(f'Document "{self.name}": write some text...\n')
        self.text = text
        print(f'Document {self.name} is rewritten: {text}')

    def show(self):
        print(f'Content of document {self.name}:\n{self.text}')

    def close(self):
        print(f'Document {self.name} is closed!')

    def delete(self):
        print(f'Document {self.name} is deleted!')
        del(self)

# инициатор команды, обращается к команде для выполнения запросы
class MenuItem:
    def __init__(self,command):
        self._command = command

    def click(self):
        self._command.execute()

# создает объект класса конкретной команды и устанавливает его получателя
class Application:
    def __init__(self):
        self.docs = {}
        self.menu_items={'open':MenuItem(OpenCommand(self)),
                         'rewrite':MenuItem(RewriteCommand(self)),
                         'del':MenuItem(DeleteCommand(self))}
        self.current_doc = None

    def add_doc(self,document):
        self.docs[hash(document.name)]=document

    def delete_doc(self,name):
        self.docs.pop(hash(name))

    def show_all_documents(self):
        res = ('All documents: ')
        res += ', '.join(list(map(lambda i: i.name, self.docs.values())))
        res += '.'
        print(res)
      
    def show_current_document(self):
      print(f'Current document: {self.current_doc.name if self.current_doc else "None"}')

if __name__=='__main__':
    # клиент создает объект конкретной команды и устанавливает для него получателя
    my_app = Application()
    command = input('PRINT COMMAND (open/exit)\n')
    while command!='exit':
        if command in my_app.menu_items.keys():
          my_app.menu_items[command].click()
          my_app.show_all_documents()
          my_app.show_current_document()
        else:
          print(f'No command {command} is found. \n')
        command = input('PRINT COMMAND (open/rewrite/del/exit)\n')
    print('Good bye!')

PRINT COMMAND (open/exit)
open
Enter the name of document...
New doc
New document New doc is created!
Document "New doc": write some text...
Text1
Document New doc is rewritten: Text1
Content of document New doc:
Text1
All documents: New doc.
Current document: New doc
PRINT COMMAND (open/rewrite/del/exit)
open
Enter the name of document...
New doc2
New document New doc2 is created!
Document "New doc2": write some text...
text2
Document New doc2 is rewritten: text2
Content of document New doc2:
text2
All documents: New doc, New doc2.
Current document: New doc2
PRINT COMMAND (open/rewrite/del/exit)
rewrite
Document "New doc2": write some text...
new text in doc2
Document New doc2 is rewritten: new text in doc2
All documents: New doc, New doc2.
Current document: New doc2
PRINT COMMAND (open/rewrite/del/exit)
exit
Good bye!
